# MIS High level

In [ ]:
import networkx as nx

# Create a simple triangle graph
graph = nx.Graph()
graph.add_edges_from([(0, 1), (0, 2)])

In [ ]:

from mis import MISInstance, MISSolver
from mis.config import SolverConfig
from mis import BackendType 

Solve using the configuration on the QUTIP backend

In [ ]:

# Create the MIS instance
instance = MISInstance(graph)

# Define classical solver configuration
config = SolverConfig(
    method = "wmis",  # or "greedy"
    use_quantum=False,               
    backend=BackendType.QUTIP,       
    max_iterations=1
)

# Run the solver
solver = MISSolver(instance, config)
result = solver.solve()

In [ ]:
# Display results
print("MIS solution:", result.bitstrings)
print("Solution cost:", result.costs)

Solve using Remote QPU backend 

In [ ]:
from mis import Connecter 
# Rename connector 
# - builds on top of the executor 

In [ ]:
conn = Connecter(backend=BackendType.REMOTE_EMUMPS,  # update the backend
    project_id="project-123",
    username="your_username",
    password="your_password")

# integrate limitations for using backends with different types of graphs 
# 15 atoms - QTips
# Add warnings about size about upper bounds - state vector emulation (15 atoms)
# emu mps ??

In [ ]:
# Point - keep the connection separate than solver configration 
# Keep the connector as an optional argument somewhere below

# Set up the quantum solver config
config = SolverConfig(
    use_quantum=True,
    method = "wmis",  # or "greedy"
    max_iterations=3,
    connector = conn
)

# Solve the problem using quantum solver
solver = MISSolver(instance, config)

# raise errors here when the register are not compatible 
result = solver.solve()

In [ ]:
# Print output
print("Quantum MIS solution:", result.bitstrings)
print("Solution cost:", result.costs)